# Langchain의 Memory 기능

**Memory**란 사용자가 대규모 언어 모델(LLM, Large Language Model)과 주고받은 대화 내용을 저장하고 이를 이후의 대화에 활용하는 기능을 말한다.

## 왜 Memory가 필요한가?

- 기본적으로 LLM은 상태 비저장(stateless) 모델이다. 
  - 의미: 한 번의 질문에 대해 답변을 제공하고, 그 이후에는 해당 질문과 답변 내용을 기억하지 못한다. 따라서 사용자가 이전 대화에 기반한 후속 질문을 하면, 모델은 맥락을 이해하지 못하고 정확한 답변을 하지 못한다.
- 이 문제를 해결하기 위해, 지금까지의 대화 내용을 저장하고 이후 질문을 할 때 함께 제공하여 맥락을 이어갈 수 있도록 하는 기능이 바로 Memory이다.

## Memory의 동작 방식

- 사용자의 질문과 LLM의 응답을 저장한다.
- 이후 사용자가 새로운 질문을 하면, **저장된 이전 대화 내용과 함께 모델에 전달**하여 자연스러운 연속 대화가 가능하도록 한다.
- **주의**:
  - 대화 이력이 너무 길어지면 현재 질문과 관련없는 내용이 많아지게 되고 이것이 noise가 되어 LLM이 부정확한 응답을 할 수도 있다. 
  - LLM마다 입력으로 받을 수 있는 [**토큰(Token)** 수에 제한](https://platform.openai.com/docs/models/compare)이 있다. 그래서 대화 내용을 무한정 저장하고 전달할 수 없다.
  - Close source llm을 사용하는 경우 대화 이력이 길어지면 그많은 많은 토큰을 입력하게 되어 비용이 늘어나게 된다.
  - 그래서 위와 같은 이유로 대화 이력을 모두 보내기 보다 최근 일부만 보내거나 요약하는 등의 관리가 필요하다.

![memory.png](figures/memory.png)


# 메시지 저장소: ChatMessageHistory
메시지 기록을 관리하는 객체로 어디에 저장느냐에 따라 여러 클래스들이 구현되어 제공된다.

## 종류
- **BaseChatMessageHistory**
    - 모든 메시지 기록 저장소 클래스의 **기본(최상위) 클래스**이다. 메시지를 저장하고 검색하는 기능을 정의하고 있으며, 이 클래스를 상속받아 다양한 저장소 방식이 구현된다.
- **InMemoryChatMessageHistory**
    - 메시지를 **메모리에 저장**하는 방식이다. 속도가 빠르지만, 프로그램을 종료하면 저장된 메시지는 사라진다.
- 외부 저장소 연동 
    - Langchain은 다양한 **3rd-party 저장소**와 연동할 수 있다. 예를 들어 SQLite, PostgreSQL, Redis, MongoDB 등을 사용해 메시지를 영구적으로 저장할 수 있다.
    - https://python.langchain.com/docs/integrations/memory/

# RunnableWithMessageHistory

- **`RunnableWithMessageHistory`**는 `ChatMessageHistory`와 `Runnable Chain`을 이용해 대화 이력을 관리하면서 대화할 수있도록 처리하는 `Runnable` 이다.
  - 대화형 애플리케이션에서는 사용자와 AI 간의 여러 번의 주고받는 대화를 통해 작업을 수행한다. 이때, 이전 대화 내용을 기억하지 못하면 일관성 없는 응답이 발생할 수 있다. 
  - `RunnableWithMessageHistory`는 이러한 문제를 해결하고 대화 흐름을 자연스럽게 유지하도록 설계되었다.

## 특징

- 체인이 실행될 때마다 **대화 메시지를 자동으로 기록**하여 개발자가 별도로 상태를 관리하지 않아도 된다.
- `session_id`(대화 ID)를 사용하여 대화를 구분한다.
  - 동일한 `session_id`를 사용하면 이전 대화를 이어갈 수 있다.
  - 새로운 `session_id`를 사용하면 새로운 대화로 인식된다.

## 생성

`RunnableWithMessageHistory`는 다음과 같은 요소들을 initializer에 전달해 생성한다.

- **runnable**: 실제 작업을 수행하는 체인(`Runnable`) 객체이다.
- **get_session_history**: 주어진 `session_id`에 해당하는 메시지 기록 저장소(`ChatMessageHistory`) 객체를 반환하는 함수이다.
- **input_messages_key**: 사용자 입력 메시지를 저장할 입력 필드의 이름이다.
- **history_messages_key**: 저장된 이전 대화 메시지를 불러올 필드의 이름이다.

이를 통해 체인을 실행할 때마다 이전 메시지가 자동으로 전달되고, 새로운 메시지도 기록된다.

[Langchain Memory Integration 문서](https://python.langchain.com/docs/integrations/memory/)

## RunnableWithMessageHistory를 이용해 Chain 구성